# LangChain Memory Chatbot with RunnableWithMessageHistory 🧠

This notebook demonstrates a modern LangChain chatbot using the **officially supported** memory approach:

- **🎯 RunnableWithMessageHistory**: Official LangChain memory management
- **🛠️ Custom dictionary tools**: For data storage and retrieval  
- **💾 Automatic persistence**: Built-in conversation history
- **👥 Multi-session support**: Separate conversations by session ID
- **🚀 Much simpler code**: Compared to manual message tracking

### 📚 **What You'll Learn:**
- How to use the official LangChain memory approach (no deprecated warnings!)
- Creating custom tools with `StructuredTool.from_function`
- Building agents with automatic conversation memory
- Managing multiple conversation sessions

In [1]:
# Import Required Libraries
import json
from typing import Dict, Any, List

# Core LangChain imports for modern approach
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.tools import StructuredTool
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Modern memory management (officially supported)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

# For tool creation
from pydantic import BaseModel

# Load configuration
with open('config.json', 'r') as f:
    config = json.load(f)
    
print("📦 All imports completed!")
print("✅ Using modern RunnableWithMessageHistory approach")
print("🚀 No deprecated components - future-proof code!")

📦 All imports completed!
✅ Using modern RunnableWithMessageHistory approach
🚀 No deprecated components - future-proof code!


In [2]:
# Core LangChain imports for modern approach
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import StructuredTool
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# For tool creation
from pydantic import BaseModel
from typing import Dict, List, Any

print("✅ All LangChain imports loaded (modern approach - no deprecated components!)")

# Custom Dictionary Manager Class
class DictionaryManager:
    """A class that manages a dictionary and provides tools for LangChain agents."""
    
    def __init__(self):
        self.data: Dict[str, Any] = {}
        
    def get_value(self, key: str) -> str:
        """Get a value from the dictionary by key."""
        try:
            if key in self.data:
                value = self.data[key]
                return f"Value for key '{key}': {value}"
            else:
                return f"Key '{key}' not found in dictionary. Available keys: {list(self.data.keys())}"
        except Exception as e:
            return f"Error getting value: {str(e)}"
    
    def update_value(self, key: str, value: str) -> str:
        """Update or add a key-value pair in the dictionary."""
        try:
            old_value = self.data.get(key, "not found")
            self.data[key] = value
            return f"Successfully updated key '{key}' from '{old_value}' to '{value}'"
        except Exception as e:
            return f"Error updating value: {str(e)}"
    
    def list_keys(self) -> str:
        """List all keys in the dictionary."""
        try:
            keys = list(self.data.keys())
            if keys:
                return f"Dictionary keys ({len(keys)}): {keys}"
            else:
                return "Dictionary is empty (no keys found)"
        except Exception as e:
            return f"Error listing keys: {str(e)}"
    
    def get_full_dict(self) -> Dict[str, Any]:
        """Get the full dictionary (for debugging)."""
        return self.data.copy()

# Create global dictionary manager instance
dict_manager = DictionaryManager()
print("🗂️  Dictionary manager created!")

✅ All LangChain imports loaded (modern approach - no deprecated components!)
🗂️  Dictionary manager created!


In [3]:
# Create LangChain Tools using StructuredTool.from_function

# Tool 1: Get value from dictionary
get_tool = StructuredTool.from_function(
    func=dict_manager.get_value,
    name="get_dictionary_value",
    description="Get a value from the dictionary using a key. Use this when the user asks to retrieve, get, or look up a value.",
)

# Tool 2: Update dictionary value
update_tool = StructuredTool.from_function(
    func=dict_manager.update_value,
    name="update_dictionary_value", 
    description="Update or add a key-value pair in the dictionary. Use this when the user wants to store, save, set, or update a value.",
)

# Tool 3: List dictionary keys
list_keys_tool = StructuredTool.from_function(
    func=dict_manager.list_keys,
    name="list_dictionary_keys",
    description="List all keys currently stored in the dictionary. Use this when the user wants to see what's available or get an overview.",
)

# Combine all tools
tools = [get_tool, update_tool, list_keys_tool]

print("🔧 Created 3 dictionary tools:")
for tool in tools:
    print(f"  • {tool.name}: {tool.description}")

print(f"\n📊 Current dictionary state: {dict_manager.get_full_dict()}")

🔧 Created 3 dictionary tools:
  • get_dictionary_value: Get a value from the dictionary using a key. Use this when the user asks to retrieve, get, or look up a value.
  • update_dictionary_value: Update or add a key-value pair in the dictionary. Use this when the user wants to store, save, set, or update a value.
  • list_dictionary_keys: List all keys currently stored in the dictionary. Use this when the user wants to see what's available or get an overview.

📊 Current dictionary state: {}


In [4]:
# Initialize LLM
llm = ChatOpenAI(
    model=config["openai"]["default_model"],
    api_key=config["openai"]["api_key"]
)

# Create in-memory message store for conversation history
store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    """Get or create conversation history for a session."""
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

print(f"🤖 LLM initialized with model: {config['openai']['default_model']}")
print("💾 Memory store created - supports multiple conversation sessions")
print("🎯 Using official RunnableWithMessageHistory approach")

🤖 LLM initialized with model: gpt-4.1
💾 Memory store created - supports multiple conversation sessions
🎯 Using official RunnableWithMessageHistory approach


In [5]:
# Create prompt template with chat history placeholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant with access to dictionary tools. You can get, update, and list dictionary keys. Use these tools when users ask about storing or retrieving information."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Create agent using the modern approach
agent = create_openai_tools_agent(llm, tools, prompt)

# Create agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 🌟 MODERN APPROACH: Wrap with RunnableWithMessageHistory for automatic memory management
agent_with_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

print("🤖 Agent created!")
print("🔧 Available tools:", [tool.name for tool in tools])
print("✨ Wrapped with RunnableWithMessageHistory for automatic memory management")
print("💬 Ready for conversation with persistent, multi-session memory!")

🤖 Agent created!
🔧 Available tools: ['get_dictionary_value', 'update_dictionary_value', 'list_dictionary_keys']
✨ Wrapped with RunnableWithMessageHistory for automatic memory management
💬 Ready for conversation with persistent, multi-session memory!


In [6]:
# Simple Helper Functions using RunnableWithMessageHistory

def chat(message: str, session_id: str = "default") -> str:
    """
    Chat with the agent using automatic memory management.
    
    Args:
        message: The user's message
        session_id: Conversation session ID (default: "default")
        
    Returns:
        str: The agent's response
    """
    try:
        response = agent_with_history.invoke(
            {"input": message},
            config={"configurable": {"session_id": session_id}}
        )
        return response["output"]
    except Exception as e:
        return f"Error: {str(e)}"

def show_memory(session_id: str = "default") -> None:
    """Display the conversation history for a session."""
    history = get_session_history(session_id)
    print(f"💭 Conversation History (Session: {session_id}):")
    print("=" * 60)
    
    if history.messages:
        for i, message in enumerate(history.messages, 1):
            if hasattr(message, 'type'):
                msg_type = message.type
                if msg_type == "human":
                    print(f"{i}. 👤 Human: {message.content}")
                elif msg_type == "ai":
                    print(f"{i}. 🤖 AI: {message.content}")
                elif msg_type == "tool":
                    print(f"{i}. 🛠️  Tool: {message.content}")
                else:
                    print(f"{i}. ❓ {msg_type}: {message.content}")
            else:
                print(f"{i}. {type(message).__name__}: {message.content}")
    else:
        print("No conversation history yet.")
    print("=" * 60)

def show_dictionary() -> None:
    """Display the current dictionary state."""
    print("🗂️  Dictionary Contents:")
    print("=" * 50)
    data = dict_manager.get_full_dict()
    if data:
        for key, value in data.items():
            print(f"  {key}: {value}")
    else:
        print("  Dictionary is empty")
    print("=" * 50)

def clear_memory(session_id: str = "default") -> None:
    """Clear the conversation memory for a session."""
    if session_id in store:
        store[session_id] = InMemoryChatMessageHistory()
    print(f"🧹 Conversation memory cleared for session: {session_id}")

def clear_dictionary() -> None:
    """Clear the dictionary."""
    dict_manager.data.clear()
    print("🗑️  Dictionary cleared!")

def list_sessions() -> None:
    """List all active conversation sessions."""
    print("📋 Active Sessions:")
    print("=" * 30)
    if store:
        for session_id, history in store.items():
            msg_count = len(history.messages)
            print(f"  • {session_id}: {msg_count} messages")
    else:
        print("  No active sessions")
    print("=" * 30)

print("✨ Simple helper functions created using RunnableWithMessageHistory:")
print("  • chat(message, session_id='default') - Chat with automatic memory")
print("  • show_memory(session_id='default') - View conversation history")
print("  • show_dictionary() - View dictionary contents")
print("  • clear_memory(session_id='default') - Clear session memory")
print("  • clear_dictionary() - Clear dictionary data")
print("  • list_sessions() - List all conversation sessions")
print("\n🎯 Much simpler code with more features!")

✨ Simple helper functions created using RunnableWithMessageHistory:
  • chat(message, session_id='default') - Chat with automatic memory
  • show_memory(session_id='default') - View conversation history
  • show_dictionary() - View dictionary contents
  • clear_memory(session_id='default') - Clear session memory
  • clear_dictionary() - Clear dictionary data
  • list_sessions() - List all conversation sessions

🎯 Much simpler code with more features!


## 🧪 Test the Chatbot

Now you can interact with the chatbot! Try these examples:

In [7]:
# Example 1: Basic introduction and storing data
response = chat("Hello! Can you help me store some information?")
print("🤖 Response:", response)



> Entering new AgentExecutor chain...
Of course! What information would you like to store? Please provide the details, and I’ll help you save them.

> Finished chain.
🤖 Response: Of course! What information would you like to store? Please provide the details, and I’ll help you save them.
Of course! What information would you like to store? Please provide the details, and I’ll help you save them.

> Finished chain.
🤖 Response: Of course! What information would you like to store? Please provide the details, and I’ll help you save them.


In [8]:
# Example 2: Store some data
response = chat("Please store my name as 'John' and my favorite color as 'blue'")
print("🤖 Response:", response)



> Entering new AgentExecutor chain...

Invoking: `update_dictionary_value` with `{'key': 'name', 'value': 'John'}`


Successfully updated key 'name' from 'not found' to 'John'
Invoking: `update_dictionary_value` with `{'key': 'favorite_color', 'value': 'blue'}`


Successfully updated key 'favorite_color' from 'not found' to 'blue'
Invoking: `update_dictionary_value` with `{'key': 'name', 'value': 'John'}`


Successfully updated key 'name' from 'not found' to 'John'
Invoking: `update_dictionary_value` with `{'key': 'favorite_color', 'value': 'blue'}`


Successfully updated key 'favorite_color' from 'not found' to 'blue'Your information has been stored:
- Name: John
- Favorite color: blue

If you'd like to save or update anything else, just let me know!

> Finished chain.
🤖 Response: Your information has been stored:
- Name: John
- Favorite color: blue

If you'd like to save or update anything else, just let me know!
Your information has been stored:
- Name: John
- Favorite color: blue

In [9]:
# Example 3: Retrieve data
response = chat("What's my name?")
print("🤖 Response:", response)



> Entering new AgentExecutor chain...

Invoking: `get_dictionary_value` with `{'key': 'name'}`


Value for key 'name': John
Invoking: `get_dictionary_value` with `{'key': 'name'}`


Value for key 'name': JohnYour name is John. If you need to update this or retrieve more information, just ask!

> Finished chain.
🤖 Response: Your name is John. If you need to update this or retrieve more information, just ask!
Your name is John. If you need to update this or retrieve more information, just ask!

> Finished chain.
🤖 Response: Your name is John. If you need to update this or retrieve more information, just ask!


In [10]:
# Example 4: List all stored data
response = chat("What information do you have stored about me?")
print("🤖 Response:", response)



> Entering new AgentExecutor chain...

Invoking: `list_dictionary_keys` with `{}`


Dictionary keys (2): ['name', 'favorite_color']
Invoking: `list_dictionary_keys` with `{}`


Dictionary keys (2): ['name', 'favorite_color']I have the following information stored about you:
- Your name
- Your favorite color

If you’d like the specific values or wish to add or update anything, just let me know!

> Finished chain.
🤖 Response: I have the following information stored about you:
- Your name
- Your favorite color

If you’d like the specific values or wish to add or update anything, just let me know!
I have the following information stored about you:
- Your name
- Your favorite color

If you’d like the specific values or wish to add or update anything, just let me know!

> Finished chain.
🤖 Response: I have the following information stored about you:
- Your name
- Your favorite color

If you’d like the specific values or wish to add or update anything, just let me know!


In [11]:
# Example 5: Multi-session support
print("🧪 Testing multi-session capability:")

# Session 1 - Alice
response1 = chat("Hi, I'm Alice and I love pizza", session_id="alice")
print(f"Alice: {response1}")

# Session 2 - Bob  
response2 = chat("Hello, I'm Bob and I enjoy coding", session_id="bob")
print(f"Bob: {response2}")

# Back to Alice's session
response3 = chat("What do I love?", session_id="alice")
print(f"Alice (asking about herself): {response3}")

# Check Bob's session
response4 = chat("What do I enjoy?", session_id="bob") 
print(f"Bob (asking about himself): {response4}")

print("\n🎯 Notice: Each session maintains its own conversation memory!")

🧪 Testing multi-session capability:


> Entering new AgentExecutor chain...
Hi Alice! It’s great to meet you. Pizza is a classic favorite. Is there a particular type or topping you love the most? Also, would you like me to remember that you love pizza for future conversations?

> Finished chain.
Alice: Hi Alice! It’s great to meet you. Pizza is a classic favorite. Is there a particular type or topping you love the most? Also, would you like me to remember that you love pizza for future conversations?


> Entering new AgentExecutor chain...
Hi Alice! It’s great to meet you. Pizza is a classic favorite. Is there a particular type or topping you love the most? Also, would you like me to remember that you love pizza for future conversations?

> Finished chain.
Alice: Hi Alice! It’s great to meet you. Pizza is a classic favorite. Is there a particular type or topping you love the most? Also, would you like me to remember that you love pizza for future conversations?


> Entering new AgentEx

## ✅ Modern LangChain Memory with RunnableWithMessageHistory

This notebook uses the **officially supported** LangChain memory approach:

- ❌ **Old (Deprecated)**: `ConversationBufferMemory` 
- ✅ **New (Official)**: `RunnableWithMessageHistory` with `InMemoryChatMessageHistory`

### Key Benefits:
- 🚀 **No deprecation warnings** - Officially supported, future-proof
- 🎯 **Much simpler code** - ~20 lines vs 100+ lines for manual approach
- 💾 **Automatic persistence** - Built-in conversation memory management
- 👥 **Multi-session support** - Separate conversations by session ID
- 🔧 **Tool compatibility** - Works seamlessly with agent tools
- 📝 **Flexible backends** - Can switch to Redis, SQLite, etc.

### How it works:
1. `InMemoryChatMessageHistory` automatically stores conversation messages
2. `RunnableWithMessageHistory` wraps the agent with memory management
3. Each session gets its own isolated conversation history
4. Memory is managed transparently - no manual message tracking needed!

### Session Management:
- Default session: `chat("Hello!")` 
- Named sessions: `chat("Hello!", session_id="user123")`
- Multiple conversations run independently

In [12]:
# View conversation memory (default session)
show_memory()

# Show all active sessions
print("\n")
list_sessions()

💭 Conversation History (Session: default):
1. 👤 Human: Hello! Can you help me store some information?
2. 🤖 AI: Of course! What information would you like to store? Please provide the details, and I’ll help you save them.
3. 👤 Human: Please store my name as 'John' and my favorite color as 'blue'
4. 🤖 AI: Your information has been stored:
- Name: John
- Favorite color: blue

If you'd like to save or update anything else, just let me know!
5. 👤 Human: What's my name?
6. 🤖 AI: Your name is John. If you need to update this or retrieve more information, just ask!
7. 👤 Human: What information do you have stored about me?
8. 🤖 AI: I have the following information stored about you:
- Your name
- Your favorite color

If you’d like the specific values or wish to add or update anything, just let me know!


📋 Active Sessions:
  • default: 8 messages
  • alice: 4 messages
  • bob: 4 messages


In [13]:
# View memory from specific sessions
print("🔍 Alice's conversation history:")
show_memory("alice")

print("\n🔍 Bob's conversation history:")
show_memory("bob")

🔍 Alice's conversation history:
💭 Conversation History (Session: alice):
1. 👤 Human: Hi, I'm Alice and I love pizza
2. 🤖 AI: Hi Alice! It’s great to meet you. Pizza is a classic favorite. Is there a particular type or topping you love the most? Also, would you like me to remember that you love pizza for future conversations?
3. 👤 Human: What do I love?
4. 🤖 AI: I don’t have any information stored yet about what you love. Would you like me to remember that you love pizza for future reference?

🔍 Bob's conversation history:
💭 Conversation History (Session: bob):
1. 👤 Human: Hello, I'm Bob and I enjoy coding
2. 🤖 AI: Hi Bob! It’s great to meet you. Coding is an awesome hobby. Is there anything specific you’d like to talk about today—perhaps a coding project, a programming language, or something you’d like to learn?
3. 👤 Human: What do I enjoy?
4. 🤖 AI: You mentioned that you enjoy coding, but I don't have that stored yet. Would you like me to save "coding" as something you enjoy? This way

In [ ]:
# View dictionary contents
show_dictionary()

🗂️  Dictionary Contents:
  name: John
  favorite_color: blue


## 💬 Interactive Chat

Use the cell below to continue chatting with the agent:

In [ ]:
# Chat with the agent - modify the message and run this cell
message = "Tell me about yourself and what you can do"
session_id = "default"  # Change this to create different conversations

response = chat(message, session_id)
print(f"👤 You ({session_id}): {message}")
print(f"🤖 Bot: {response}")

# Optional: Show current memory state
print(f"\n📊 Current session ({session_id}) has {len(get_session_history(session_id).messages)} messages")



> Entering new AgentExecutor chain...
I’m an AI assistant designed to help you with a wide range of tasks! Here’s a little about what I can do for you:

1. Store and Retrieve Information: I can save important details for you (like names, favorite things, reminders, and more) and recall them whenever you ask.
2. Dictionary & Organization: I can help you organize personal notes, keep track of lists, or remember specific pieces of information.
3. Answer Questions: I can answer general knowledge questions, explain concepts, or help you understand things.
4. Productivity Boost: Need help planning, setting reminders, or managing to-dos? I can help you structure your schedule and tasks.
5. Friendly Conversation: If you just want to chat, I’m here for that too!

Is there something specific you’d like to know or try first?

> Finished chain.
👤 You: Tell me about yourself and what you can do
🤖 Bot: I’m an AI assistant designed to help you with a wide range of tasks! Here’s a little about what 

## 📚 Legacy Examples (For Educational Comparison)

The following sections show alternative approaches we explored, kept here for educational value:

### 🔍 **What These Show:**
- **Manual tool tracking** - How to detect tool usage by monitoring state changes
- **Custom callback handlers** - Advanced technique for capturing tool interactions  
- **Complex implementations** - What we replaced with the simpler RunnableWithMessageHistory

### 💡 **Key Takeaway:**
The simple `RunnableWithMessageHistory` approach above achieves the same goals with much less code!

In [ ]:
# Test enhanced tool tracking
print("🧪 Testing enhanced tool tracking:")
print("Dictionary before:", dict_manager.get_full_dict())

response = chat_with_tool_tracking("Please store my age as 25 and my city as 'San Francisco'")
print(f"\n🤖 Response: {response}")

print("\nDictionary after:", dict_manager.get_full_dict())

🧪 Testing enhanced tool tracking:
Dictionary before: {'name': 'John', 'favorite_color': 'blue'}


> Entering new AgentExecutor chain...

Invoking: `update_dictionary_value` with `{'key': 'age', 'value': '25'}`


Successfully updated key 'age' from 'not found' to '25'
Invoking: `update_dictionary_value` with `{'key': 'city', 'value': 'San Francisco'}`


Successfully updated key 'city' from 'not found' to 'San Francisco'
Invoking: `update_dictionary_value` with `{'key': 'age', 'value': '25'}`


Successfully updated key 'age' from 'not found' to '25'
Invoking: `update_dictionary_value` with `{'key': 'city', 'value': 'San Francisco'}`


Successfully updated key 'city' from 'not found' to 'San Francisco'Your information has been updated:
- Age: 25
- City: San Francisco

Let me know if you'd like to store more details or retrieve any of your stored information!

> Finished chain.

🤖 Response: Your information has been updated:
- Age: 25
- City: San Francisco

Let me know if you'd like to sto

In [ ]:
# Advanced Approach: Custom Callback Handler for Complete Tool Tracking

from langchain.callbacks.base import BaseCallbackHandler
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from typing import Any, Dict, List

class MemoryTrackingCallback(BaseCallbackHandler):
    """Custom callback handler that captures all tool calls and responses."""
    
    def __init__(self):
        self.current_messages = []
        self.tool_calls = []
        self.tool_responses = []
    
    def on_agent_action(self, action, **kwargs):
        """Called when the agent takes an action (calls a tool)."""
        self.tool_calls.append({
            'tool': action.tool,
            'input': action.tool_input,
            'log': action.log
        })
    
    def on_tool_end(self, output, **kwargs):
        """Called when a tool finishes executing."""
        self.tool_responses.append(str(output))
    
    def get_tool_messages(self):
        """Generate ToolMessage objects from captured tool calls and responses."""
        messages = []
        for i, (call, response) in enumerate(zip(self.tool_calls, self.tool_responses)):
            # Create AI message with tool call
            ai_msg = AIMessage(
                content=f"Using tool: {call['tool']}",
                additional_kwargs={
                    'tool_calls': [{
                        'function': {
                            'name': call['tool'],
                            'arguments': str(call['input'])
                        }
                    }]
                }
            )
            messages.append(ai_msg)
            
            # Create tool message with response
            tool_msg = ToolMessage(
                content=response,
                tool_call_id=f"call_{i}"
            )
            messages.append(tool_msg)
        return messages
    
    def reset(self):
        """Reset the callback for the next interaction."""
        self.current_messages = []
        self.tool_calls = []
        self.tool_responses = []

# Create global callback instance
memory_callback = MemoryTrackingCallback()

def chat_with_full_tracking(message: str) -> str:
    """
    Advanced chat function that captures ALL intermediate steps including tool calls.
    Uses a custom callback handler to track every tool interaction.
    """
    try:
        # Reset callback for this interaction
        memory_callback.reset()
        
        # Add user message to history
        user_message = HumanMessage(content=message)
        chat_history.append(user_message)
        
        # Invoke agent with callback to capture tool interactions
        response = agent_executor.invoke(
            {
                "input": message,
                "chat_history": chat_history[:-1]
            },
            {"callbacks": [memory_callback]}
        )
        
        # Add all the captured tool messages to chat history
        tool_messages = memory_callback.get_tool_messages()
        chat_history.extend(tool_messages)
        
        # Add final AI response
        ai_message = AIMessage(content=response["output"])
        chat_history.append(ai_message)
        
        return response["output"]
        
    except Exception as e:
        return f"Error: {str(e)}"

def show_complete_memory():
    """Show the complete conversation including all tool interactions."""
    print("💭 Complete Conversation History (with Full Tool Tracking):")
    print("=" * 80)
    
    if not chat_history:
        print("No conversation history yet.")
        print("=" * 80)
        return
    
    for i, message in enumerate(chat_history, 1):
        if isinstance(message, HumanMessage):
            print(f"{i}. 👤 User: {message.content}")
        elif isinstance(message, AIMessage):
            # Check for tool calls in additional_kwargs
            if hasattr(message, 'additional_kwargs') and 'tool_calls' in message.additional_kwargs:
                tool_calls = message.additional_kwargs['tool_calls']
                print(f"{i}. 🤖 AI (Tool Call): {message.content}")
                for tc in tool_calls:
                    func = tc.get('function', {})
                    print(f"     🔧 Calling: {func.get('name', 'unknown')}({func.get('arguments', 'no args')})")
            else:
                print(f"{i}. 🤖 AI: {message.content}")
        elif isinstance(message, ToolMessage):
            print(f"{i}. 🛠️  Tool Response: {message.content}")
        else:
            print(f"{i}. ❓ Other ({type(message).__name__}): {message.content}")
    
    print("=" * 80)

print("🚀 Advanced tool tracking system loaded!")
print("📝 New functions available:")
print("  • chat_with_full_tracking(message) - Complete tool call capture")
print("  • show_complete_memory() - Display full conversation with tool details")
print("\n💡 This captures the actual tool calls and responses in the message history!")

🚀 Advanced tool tracking system loaded!
📝 New functions available:
  • chat_with_full_tracking(message) - Complete tool call capture
  • show_complete_memory() - Display full conversation with tool details

💡 This captures the actual tool calls and responses in the message history!


In [ ]:
# Test the advanced tool tracking
print("🧪 Testing advanced tool tracking with callback handler:")
print("Current chat history length:", len(chat_history))

# Clear history for clean test
clear_memory()

response = chat_with_full_tracking("Please store my profession as 'Developer' and check what keys are available")
print(f"\n🤖 Response: {response}")

print(f"\n📊 Chat history now has {len(chat_history)} messages")
show_complete_memory()

🧪 Testing advanced tool tracking with callback handler:
Current chat history length: 12
🧹 Conversation memory cleared!


> Entering new AgentExecutor chain...

Invoking: `update_dictionary_value` with `{'key': 'profession', 'value': 'Developer'}`


Successfully updated key 'profession' from 'not found' to 'Developer'
Invoking: `list_dictionary_keys` with `{}`


Dictionary keys (5): ['name', 'favorite_color', 'age', 'city', 'profession']
Invoking: `update_dictionary_value` with `{'key': 'profession', 'value': 'Developer'}`


Successfully updated key 'profession' from 'not found' to 'Developer'
Invoking: `list_dictionary_keys` with `{}`


Dictionary keys (5): ['name', 'favorite_color', 'age', 'city', 'profession']Your profession has been stored as "Developer".

Currently, the available keys in your dictionary are:
- name
- favorite_color
- age
- city
- profession

Let me know if you'd like to update or retrieve any other information!

> Finished chain.

🤖 Response: Your profession has be

## ✅ Using the Official RunnableWithMessageHistory Approach

This notebook now uses the **officially recommended** `RunnableWithMessageHistory` approach!

### ✅ **Current Approach (What We're Using):**
- **RunnableWithMessageHistory** with `InMemoryChatMessageHistory`
- ✅ Automatic message management
- ✅ Built-in persistence options 
- ✅ Multi-session support
- ✅ Much less code!
- ✅ Officially supported (no deprecation warnings)

### 📊 **Code Comparison:**

| Approach | Lines of Code | Complexity | Features |
|----------|--------------|------------|----------|
| ❌ **Manual Tracking** | ~100+ lines | Medium | Basic chat history |
| ✅ **RunnableWithMessageHistory** (This notebook) | ~30 lines | Low | Auto-persistence, multi-session |
| 🚀 **LangGraph** | ~50-80 lines | High | State management, workflows, checkpoints |

### 🎯 **Why This Approach is Perfect:**
- **Officially supported** by LangChain team
- **Simple to understand** - great for learning
- **Production ready** - handles real-world requirements  
- **Extensible** - can switch to persistent backends (Redis, SQLite, etc.)

In [ ]:
# Example: RunnableWithMessageHistory (Much Simpler!)

from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

# Create a simple in-memory message store
store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# Wrap our agent executor with automatic message history
agent_with_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

def simple_chat(message: str, session_id: str = "default") -> str:
    """Simple chat using official RunnableWithMessageHistory - much less code!"""
    try:
        response = agent_with_history.invoke(
            {"input": message},
            config={"configurable": {"session_id": session_id}}
        )
        return response["output"]
    except Exception as e:
        return f"Error: {str(e)}"

def show_simple_history(session_id: str = "default"):
    """Show the automatically managed conversation history."""
    history = get_session_history(session_id)
    print(f"💭 Auto-Managed History (Session: {session_id}):")
    print("=" * 60)
    
    if history.messages:
        for i, message in enumerate(history.messages, 1):
            if hasattr(message, 'type'):
                msg_type = message.type
                if msg_type == "human":
                    print(f"{i}. 👤 Human: {message.content}")
                elif msg_type == "ai":
                    print(f"{i}. 🤖 AI: {message.content}")
                elif msg_type == "tool":
                    print(f"{i}. 🛠️  Tool: {message.content}")
                else:
                    print(f"{i}. ❓ {msg_type}: {message.content}")
            else:
                print(f"{i}. {type(message).__name__}: {message.content}")
    else:
        print("No conversation history yet.")
    print("=" * 60)

print("✨ RunnableWithMessageHistory setup complete!")
print("📝 Much simpler functions:")
print("  • simple_chat(message, session_id='default') - Automatic memory management!")
print("  • show_simple_history(session_id='default') - View auto-managed history")
print("\n🎯 This is the official 'simple but modern' approach!")

# ✅ RunnableWithMessageHistory Implementation Summary

print("🎯 This notebook now uses RunnableWithMessageHistory throughout!")
print("\n📋 Key components we're using:")
print("  • RunnableWithMessageHistory - Automatic memory wrapper")
print("  • InMemoryChatMessageHistory - Message storage") 
print("  • Multi-session support - Isolated conversations")
print("  • Simple helper functions - Easy to use API")

print(f"\n📊 Current setup:")
print(f"  • Memory store: {len(store)} sessions")
print(f"  • Agent: {type(agent_with_history).__name__}")
print(f"  • Tools: {len(tools)} custom tools")

print("\n✨ Benefits achieved:")
print("  🚀 No deprecation warnings")
print("  🎯 Much simpler code") 
print("  💾 Automatic persistence")
print("  👥 Multi-session support")
print("  🔧 Official LangChain approach")

✨ RunnableWithMessageHistory setup complete!
📝 Much simpler functions:
  • simple_chat(message, session_id='default') - Automatic memory management!
  • show_simple_history(session_id='default') - View auto-managed history

🎯 This is the official 'simple but modern' approach!
🎯 This notebook now uses RunnableWithMessageHistory throughout!

📋 Key components we're using:
  • RunnableWithMessageHistory - Automatic memory wrapper
  • InMemoryChatMessageHistory - Message storage
  • Multi-session support - Isolated conversations
  • Simple helper functions - Easy to use API

📊 Current setup:
  • Memory store: 0 sessions
  • Agent: RunnableWithMessageHistory
  • Tools: 3 custom tools

✨ Benefits achieved:
  🚀 No deprecation warnings
  🎯 Much simpler code
  💾 Automatic persistence
  👥 Multi-session support
  🔧 Official LangChain approach


## 🚀 Beyond RunnableWithMessageHistory: LangGraph

Our notebook now uses the **officially recommended** approach, but LangGraph offers even more advanced features:

### 📊 **Complexity vs Features:**

| Approach | Status | Lines of Code | Use Case |
|----------|--------|---------------|----------|
| ✅ **RunnableWithMessageHistory** | **← WE ARE HERE** | ~30 lines | Most applications, learning, production-ready |
| 🚀 **LangGraph** | Advanced option | ~50-80 lines | Complex workflows, enterprise applications |

### 🎯 **Current Approach (RunnableWithMessageHistory) is Perfect For:**
- ✅ Learning LangChain concepts
- ✅ Most chatbot applications  
- ✅ Production use cases
- ✅ Multi-user applications
- ✅ Simple to moderate complexity

### 🚀 **When to Consider LangGraph:**
- 🌳 **Complex workflows**: Multi-step processes, branching conversations
- 🔄 **State management**: Store more than just messages (user profiles, context, etc.)
- ⏰ **Time travel**: Need to rewind/replay conversations
- 🎛️ **Custom control**: Highly specific orchestration requirements
- 🏢 **Enterprise scale**: Very large applications with complex requirements

### 💡 **Recommendation:**
- **Stick with RunnableWithMessageHistory** for most use cases (including this notebook)
- **Consider LangGraph** only when you need its advanced features

In [ ]:
# LangGraph Example (would need: pip install langgraph)
# This is what the LangGraph approach would look like:

"""
from langgraph.graph import StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver

# Define the agent function
def call_agent(state: MessagesState):
    response = agent_executor.invoke({"input": state["messages"][-1].content})
    return {"messages": [AIMessage(content=response["output"])]}

# Create the graph
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_agent)
workflow.set_entry_point("agent")
workflow.set_finish_point("agent")

# Add memory/checkpointing
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# Usage
def langgraph_chat(message: str, thread_id: str = "1"):
    config = {"configurable": {"thread_id": thread_id}}
    response = app.invoke(
        {"messages": [HumanMessage(content=message)]}, 
        config=config
    )
    return response["messages"][-1].content
"""

print("📋 LangGraph approach shown above (commented out - needs langgraph package)")
print("🎯 Key differences:")
print("   • State management: Can store ANY data, not just messages")
print("   • Workflows: Define complex conversation flows")  
print("   • Checkpointing: Save/restore conversation state")
print("   • Backends: Persistent storage (SQLite, Postgres, etc.)")
print("   • Time travel: Go back to any conversation state")

print("\n🤔 Should you use LangGraph?")
print("   ✅ YES if: Complex workflows, production app, multi-user")
print("   ❌ NO if: Simple chatbot, learning, prototyping")

In [ ]:
# Test the simple RunnableWithMessageHistory approach
print("🧪 Testing the much simpler official approach:")

# Test conversation
response1 = simple_chat("Hi! Please store my hobby as 'photography'")
print(f"🤖 Response 1: {response1}")

response2 = simple_chat("What hobby did I just tell you about?")
print(f"🤖 Response 2: {response2}")

print(f"\n📊 Auto-managed conversation history:")
show_simple_history()

print("\n🎯 Notice: MUCH less code, automatic persistence, same functionality!")

## 🎉 Refactoring Complete!

### ✅ **What We Achieved:**

**Before (Manual Approach):**
- ❌ ~100+ lines of complex code
- ❌ Manual message tracking  
- ❌ No built-in persistence
- ❌ Single conversation only
- ❌ Potential for bugs in memory management

**After (RunnableWithMessageHistory):**
- ✅ ~30 lines of simple code  
- ✅ Automatic memory management
- ✅ Built-in persistence support
- ✅ Multi-session conversations
- ✅ Officially supported approach
- ✅ No deprecation warnings

### 🚀 **Key Features Now Available:**
1. **Automatic Memory**: No manual message tracking needed
2. **Multi-Session**: Isolated conversations (`session_id` parameter)
3. **Simple API**: Easy-to-use helper functions
4. **Tool Integration**: Seamless tool calls with memory
5. **Production Ready**: Official LangChain approach
6. **Extensible**: Can switch to persistent backends (Redis, SQLite, etc.)

### 💡 **Next Steps:**
- ✅ Use `chat(message, session_id)` for conversations
- ✅ Use `show_memory(session_id)` to inspect history  
- ✅ Use `list_sessions()` to manage multiple conversations
- 🚀 Consider LangGraph only for complex workflow requirements

**This is now a clean, modern, production-ready LangChain memory implementation!** 🎯